# TED Talk Transcripts

## Working Script for Soup

In [50]:
import glob, re, csv
from bs4 import BeautifulSoup as soup                                                     

the_file = "/Users/john/Code/tedtalks/test/transcript?language=en.0"
holding = soup(open(the_file).read(), "lxml")
at = holding.find("title").text
author = at[0:at.find(':')]
title  = at[at.find(":")+1 : at.find("|") ]
date = re.sub('[^a-zA-Z0-9]',' ', holding.select_one("span.meta__val").text)
length_data = holding.find_all('data', {'class' : 'talk-transcript__para__time'})
(m, s) = ([x.get_text().strip("\n\r") 
      for x in length_data if re.search(r"(?s)\d{2}:\d{2}", 
                                        x.get_text().strip("\n\r"))][-1]).split(':')
length = int(m) * 60 + int(s)
firstpass = re.sub(r'\([^)]*\)', '', holding.find('div', class_ = 'talk-transcript__body').text)
text = re.sub('[^a-zA-Z\.\']',' ', firstpass)
data = [str(author), str(title)]
# print(data)
with open("./output.csv", "w", newline = "") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        for item in data:
            writer.writerow(item)

## Working Bit to Read a Directory into a List

In [8]:
file_list = glob.glob('/Users/john/Code/tedtalks/test/*') # produces list
print(file_list)

['/Users/john/Code/tedtalks/test/transcript?language=en.0', '/Users/john/Code/tedtalks/test/transcript?language=en.1', '/Users/john/Code/tedtalks/test/transcript?language=en.2']


## This Should Work... (but it doesn't)

In [21]:
# Script to read 2113 texts and write them to a single csv 
# file with metadata for AUTHOR, TITLE, DATE, LENGTH.

import glob, re, csv
from bs4 import BeautifulSoup as soup                                                     

file_list = glob.glob('/Users/john/Code/tedtalks/test/*') # produces list

# Now to walk through list
for the_file in file_list:
    the_list = []
    holding = soup(open(the_file).read(), "lxml")
    at = holding.find("title").text
    author = at[0:at.find(':')]
    title  = at[at.find(":")+1 : at.find("|") ]
    date = re.sub('[^a-zA-Z0-9]',' ', holding.select_one("span.meta__val").text)
    length_data = holding.find_all('data', {'class' : 'talk-transcript__para__time'})
    (m, s) = ([x.get_text().strip("\n\r") 
          for x in length_data if re.search(r"(?s)\d{2}:\d{2}", 
                                            x.get_text().strip("\n\r"))][-1]).split(':')
    length = int(m) * 60 + int(s)
    firstpass = re.sub(r'\([^)]*\)', '', holding.find('div', class_ = 'talk-transcript__body').text)
    text = re.sub('[^a-zA-Z\.\']',' ', firstpass)
    the_list = the_list.append(author)
    print(the_list)

None
None
None


## Working Script

This is available in the repo as `text_to_csv.py`.

In [65]:
import re
import csv
import os
from bs4 import BeautifulSoup


def parse(soup):
    # both title and author are can be parsed in separate tags.
    author = soup.select_one("h4.h12.talk-link__speaker").text
    title = soup.select_one("h4.h9.m5").text
    # just need to strip the text from the date string, no regex needed.
    date = soup.select_one("span.meta__val").text.strip()      
    # we want the last time which is the talk-transcript__para__time previous to the footer.
    mn, sec = map(int, soup.select_one("footer.footer").find_previous("data", {
        "class": "talk-transcript__para__time"}).text.split(":"))
    length = (mn * 60 + sec)        
    # to ignore (Applause) etc.. we can just pull from the actual text fragment checking for (
    text = " ".join(d.text for d in soup.select("span.talk-transcript__fragment") if not d.text.startswith("("))        
    # clean the text
    text = re.sub('[^a-zA-Z\.\']', ' ', text)
    return  author.strip().encode("utf-8"), title.strip().encode("utf-8"), date, length, text


def to_csv(pth, out):
    # open file to write to.
    with open(out, "w") as out:
        # create csv.writer. 
        wr = csv.writer(out)
        # write our headers.
        wr.writerow(["author", "title", "date", "length", "text"])
        # get all our html files.
        for html in os.listdir(pth):
            with open(os.path.join(pth, html)) as f:
                # parse the file and write the data to a row.
                wr.writerow(parse(BeautifulSoup(f, "lxml")))


to_csv("./talks","output_talk.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte